In [39]:
import random
import math
from sympy import mod_inverse


# Function to compute large powers a^g modulo N

def fast(a,g,N):
    g = bin(g)         # Turn to binary
    g = g[2:]          # Remove the 0b at the beginning

    d = 1
    x = a
    for i in range (len(g)-1,-1,-1):
        if g[i]==str(1):
            d = (d * x) % N
        x = (x ** 2) % N
    return d


# Miller Rabin Primality Test

def millerRabin(n):
    s = n-1
    t = 0
    while (s&1) == 0:
        s = s//2
        t +=1
    k = 0
    while k<50:
        a = random.randrange(2,n-1)
        #a^s is computationally infeasible.  we need a more intelligent approach
        #v = (a**s)%n
        #python's core math module can do modular exponentiation
        v = fast(a,s,n) #where values are (num,exp,mod)
        if v != 1:
            i=0
            while v != (n-1):
                if i == t-1:
                    return False
                else:
                    i = i+1
                    v = (v**2)%n
        k+=2
    return True


def isMRPrime(n):
    if (n >= 3):
        if (n&1 != 0):
            if (millerRabin(n)==True):
                return True
    return False


# This is our main function to generate the primes

def generateLargePrime(k):
    # k is the desired bit length
    # primeType is the type of prime we want to generate ( Fermat, MillerRabin, Safe )
    tries=int(1000*(math.log(k,2)+1)) # number of max attempts
    total = tries
    while tries>0:
        n = random.randrange(2**(k-1),2**(k))
        tries= tries - 1
        if isMRPrime(n) == True:
            return n
    return 0


def φ(N,p,q):          # Function to compute (p-1)*(q-1), N parameter only to show we compute φ(Ν) as we already know p and q
    return (p-1)*(q-1)


# Function to compute d knowing e and φ(N)

def modInverse(a, m) :           # Modular multiplicative inverse function from:
    a = a % m;                   # https://www.geeksforgeeks.org/multiplicative-inverse-under-modulo-m/
    print(a)
    for x in range(1, m) : 
        if ((a * x) % m == 1) : 
            return x 
    return 1

    
def generateE(f):
    for i in range (10000):
        e = random.randrange(3,f-1)        # 1 < e < φ(N)
        if math.gcd(e,f)==1:
            return e
    return 0
    
    
# KEY GENERATING FUNCTION (No.1)
    
def generateRSAkeys(k):      # k is the bit length of the p and q we demand, I will use 1024 bit numbers
    print ("\nGenerating p 1024-bit prime:\n")   
    p = generateLargePrime(k)
    print(p)
    
    print ("\nGenerating q 1024-bit prime:\n")   
    q = generateLargePrime(k)
    print(q)
    
    N=p*q
    print("\nGenerating N=p*q:\n")
    print(N)
    
    f = φ(N,p,q)
    print("\nGenerating e:\n")
    e = generateE(f)
    print(e)
    
    print("\nGenerating d:\n")
    d = mod_inverse(e,f)
    print(d)
    
    print("\nGenerating dp:\n")
    dp = (mod_inverse(e,p-1))
    print(dp)
    
    print("\nGenerating dq:\n")
    dq = (mod_inverse(e,q-1))
    print(dq)
    
    print("\nGenerating qinv:\n")
    qinv = (mod_inverse(q,p))
    print(qinv)
    
    return p, q, N, e, d, dp, dq, qinv


# RSA ENCRYPTION FUNCTION (No.2)
    
def RSAencryption(message):
    return fast(message,e,N) 

# CRT-RSA DECRYPTION FUNCTION (No.3)

def CRT_RSAdecryption(encrypted,p,q,dp,dq,qinv):
    Sp = fast(encrypted,dp,p)
    Sq = fast(encrypted,dq,q)
    S = Sq + q * ((qinv*(Sp-Sq))%p)
    return S%N

# ------------------------------------------------------------------------------    
    
message = 3076

p, q, N, e, d, dp, dq, qinv= generateRSAkeys(1024) # No.1
print("\nInitial message:\n\n" + str(message))

encrypted = RSAencryption(message) # No.2
print("\nEncrypted message using RSA:\n\n" + str(encrypted))

decrypted = CRT_RSAdecryption(encrypted,p,q,dp,dq,qinv) # No.3
print("\nDecrypted message using CRT-RSA:\n\n" + str(decrypted))





Generating p 1024-bit prime:

171409048229576090352218482598100667066045705010081181929796304146937394204207655884586791223762374260489063354459056738700164912446821562189547126621060768555810733730139133857011058463084093712616336040029940018689819602010173100064809636842193075427105505737933631599803943595531132417932710470693457821593

Generating q 1024-bit prime:

159661251260983305567003597597232422563191392448062229621163880763910305784869541157861014485442324179656524531696592951173081762206629654185826444376466140637489208852394052921931024307234167693143801994947821450717234873198094163280411928721272297852276078872042026965625684328312841336003747459674801225667

Generating N=p*q:

2736738311778835379637294737292224253654217328665106574520095643961234452717214928550784542889198815407384552306978900256488031749446115324716571462267677421910126753092808648799825454897872144868221971897134719131848391372000585265946971289782134013371384712260249202444098107680475432803252234432